# Framework tests

## Sonification class interactive tests

In [ ]:
import pandas as pd
import panson as ps
import os
import time

FILE_DIR = '../media/files'
OUT_DIR = os.path.join(FILE_DIR, 'processed')

In [ ]:
df = pd.read_csv(os.path.join(OUT_DIR, "phone.csv"), sep=r',\s*', engine='python')
df.head()

In [ ]:
import sc3nb as scn

In [ ]:
# start scsynth
sc = scn.startup(start_sclang=False)
# sc = scn.startup()

# connect scsynth to the system playback
!jack_connect "SuperCollider:out_1" "system:playback_1"
!jack_connect "SuperCollider:out_2" "system:playback_2"

# multiclient setup by default
sc.server.dump_tree()

In [ ]:
sc.exit()

In [ ]:
from sc3nb.osc.osc_communication import Bundler

# message style (explicit ID allocation)
class AU04ContinuousSonification(ps.Sonification):
    
    def initialize(self):
        bundler = Bundler()
        bundler.add(
            0,
            "/d_load",
            ['/home/michele/Desktop/Thesis/tools/sc3nb/src/sc3nb/resources/synthdefs/s2.scsyndef']
        )
        return bundler
    
    def start(self):
        bundler = Bundler()
        self.au4_node_id = self._s.node_ids.allocate(1)[0]        
        bundler.add(0, "/s_new", ["s2", self.au4_node_id, 0, 0, "amp", 0])
        return bundler
    
    def stop(self, server):
        bundler = Bundler()
        self._s.node_ids.free([self.au4_node_id])    # actually this does nothing
        bundler.add(0, "/g_freeAll", [0])
        return bundler
    
    def process(self, row):
        bundler = Bundler()
        # only "max" should be enough (to clip the top part to 0.3)
        amp = scn.linlin(row["AU04_r"], 0, 1, 0, 0.3, "minmax")
        # map the intensity of the AU in one octave range
        freq = scn.midicps(scn.linlin(row["AU04_r"], 0, 5, 69, 81))
        bundler.add(0, "/n_set", [self.au4_node_id, "amp", amp, "freq", freq])
        return bundler

In [ ]:
from sc3nb.osc.osc_communication import Bundler

# message style (explicit ID allocation)
class AU04ContinuousSonification(ps.Sonification):
    
    def initialize(self):
        with Bundler(send_on_exit=False) as bundler:
            self._s.msg(
                "/d_load",
                ['/home/michele/Desktop/Thesis/tools/sc3nb/src/sc3nb/resources/synthdefs/s2.scsyndef'],
                bundle=True
            )
        return bundler
    
    def start(self):
        with Bundler(send_on_exit=False) as bundler:
            self.au4_node_id = self._s.node_ids.allocate(1)[0]
            self._s.msg("/s_new", ["s2", self.au4_node_id, 0, 0, "amp", 0], bundle=True)
        return bundler
    
    def stop(self):
        with Bundler(send_on_exit=False) as bundler:
            self._s.node_ids.free([self.au4_node_id])    # actually this does nothing
            self._s.msg("/g_freeAll", [0], bundle=True)
        return bundler
    
    def process(self, row):
        with Bundler(send_on_exit=False) as bundler:
            # only "max" should be enough (to clip the top part to 0.3)
            amp = scn.linlin(row["AU04_r"], 0, 1, 0, 0.3, "minmax")
            # map the intensity of the AU in one octave range
            freq = scn.midicps(scn.linlin(row["AU04_r"], 0, 5, 69, 81))
            self._s.msg("/n_set", [self.au4_node_id, "amp", amp, "freq", freq], bundle=True)
        return bundler

In [ ]:
from sc3nb.osc.osc_communication import Bundler
from sc3nb import SynthDef

# a (implicit ID allocation)
class AU04ContinuousSonification(ps.Sonification):
    
    # TODO: server arg is necessary?
    def initialize(self):
        with Bundler(send_on_exit=False) as bundler:
            # self._s.load_synthdefs()
            SynthDef.load("/home/michele/Desktop/Thesis/tools/sc3nb/src/sc3nb/resources/synthdefs/s2.scsyndef")
        
        return bundler

    def start(self):
        with Bundler(send_on_exit=False) as bundler:
            self.synth = scn.Synth("s2", {"amp": 0})
        
        return bundler
    
    def stop(self):
        with Bundler(send_on_exit=False) as bundler:
            self._s.free_all()
        
        return bundler
    
    def process(self, row):
        with Bundler(send_on_exit=False) as bundler:
            self.synth.set(
                # only "max" should be enough (to clip the top part to 0.3)
                "amp", scn.linlin(row["AU04_r"], 0, 1, 0, 0.3, "minmax"),
                # map the intensity of the AU in one octave range
                "freq", scn.midicps(scn.linlin(row["AU04_r"], 0, 5, 69, 81))
            )
        
        return bundler

In [ ]:
son = AU04ContinuousSonification()

**sc.server.bundler vs Bundler**
* sc.server.bundler: adds server.latency to the timestamp (also the absolute one)
* Bundler: does not add anything to the timestamp

### Realtime usage

In [ ]:
sc.server.latency = 0.1

#### Offline

In [ ]:
dp = ps.DataPlayer(son, df)
dp   # TODO: display

In [ ]:
dp.play_test()

#### Online

### NRT usage

In [ ]:
dp.export("score.wav")

## Thread behaviour

The threads outputs results in the lastly executed cell.

The cell return when the main thread finishes.

In [ ]:
import threading
import time

def worker():
    print(threading.current_thread().getName(), 'Starting')
    time.sleep(10)
    print(threading.current_thread().getName(), 'Exiting')


def my_service():
    print(threading.current_thread().getName(), 'Starting')
    time.sleep(5)
    print(threading.current_thread().getName(), 'Exiting')


t = threading.Thread(name='my_service', target=my_service)
w = threading.Thread(name='worker', target=worker)
w2 = threading.Thread(target=worker)  # use default name

w.start()
w2.start()
t.start()

# # wait until threads terminate
# w.join()
# w2.join()
# t.join()

In [ ]:
print('test')

## Misc

Save frame information as csv as expected by the videoviewer

In [ ]:
timestamps = df.timestamp.to_numpy()
np.savetxt('phone.avi.csv', timestamps, delimiter=',')

### Video

#### ipywidgets

In [ ]:
from ipywidgets import Video
video = Video.from_file('NRT_videos/phone-processed-son.mp4')
video

In [ ]:
from ipywidgets import Image
image = Image()

#### PyQt

The following cell enable PyQt5 event loop integration. This is done by opening a QApplication for your notebook. This means that we have to avoid creating QApplication objects in our code (and consequenctly to start the event loop).
* https://stackoverflow.com/questions/30606462/closing-a-pyqt-widget-in-ipython-notebook-without-using-sys-exit

We can run the following magic as many times as we want, but we have to avoid creating QApplications in our code.

In [ ]:
%gui qt5

from PyQt5 import QtWidgets, QtCore
import pyqtgraph as pg

In [ ]:
QtCore.QCoreApplication.instance()

The following disables IPython GUI event loop integration.

In [ ]:
# %gui

In [ ]:
# # GUI initialization
# window = QtWidgets.QWidget()

# imggv = pg.GraphicsView()
# viewbox = pg.ViewBox()
# viewbox.setAspectLocked()
# viewbox.invertY(True)

# imggv.setCentralItem(viewbox)

# img = pg.ImageItem(np.zeros((100, 100, 3)))  # Todo: 3 -> channel variable
# viewbox.addItem(img)

# layout = QtWidgets.QGridLayout()
# layout.addWidget(imggv, 0, 0, 4, 4)
# window.setLayout(layout)

# # window always on top
# window.setWindowFlag(QtCore.Qt.WindowStaysOnTopHint)

# window.show()

In [ ]:
from videoviewer import NBVideoViewer

vv = NBVideoViewer()
vv.start()

In [ ]:
import skvideo
import skvideo.io

frames = skvideo.io.vread('files/processed/phone.avi')
frames.shape

In [ ]:
# frame = frames[0, ::5, ::5, [0,1,2]].T
# img.setImage(frame)

In [ ]:
import ipywidgets as widgets

In [ ]:
au4_node_id = 1234
# instantiate synths
sc.server.msg("/s_new", ["s2", au4_node_id, 0, 0, "amp", 0])

In [ ]:
sc.server.free_all()

In [ ]:
max_ = 332

# out = widgets.Output()

def on_change(change):
    # with out:
    #    print(change.new)
    
    idx = change.new
    
    row = df.iloc[idx]

    # only "max" should be enough (to clip the top part to 0.3)
    amp = scn.linlin(row["AU04_r"], 0, 1, 0, 0.3, "minmax")   # TODO: exponential mapping
    # map the intensity of the AU in one octave range
    freq = scn.midicps(scn.linlin(row["AU04_r"], 0, 5, 69, 81))

    with sc.server.bundler(0.1) as bundler:
        sc.server.msg("/n_set", [au4_node_id, "amp", amp, "freq", freq], bundle=True)

    frame = frames[idx, ::5, ::5, [0,1,2]]
    vv.update(frame)
    
def on_play_stop(change):
    
    # when the video is paused
    if change.now == False:
        # silence all the synths
        with sc.server.bundler() as bundler:
            sc.server.msg("/n_set", [au4_node_id, "amp", 0], bundle=True)
    
        
slider = widgets.IntSlider(max=max_)

# can't play data with unregolar sampling rate
play = widgets.Play(
    max=max_,
    step=1,
    interval=33,
    disabled=False
)

play.observe(on_change, names='value')
play.observe(on_play_stop, names='_playing')

widgets.jslink((play, 'value'), (slider, 'value'))
widgets.HBox([play, slider])

In [ ]:
def sonify_row(df_row):
    pass

In [ ]:
def sonification_callback(event):
    frame_idx = event.cls.value
    
#     sonify_row(df.iloc[frame_idx])
    img.setImage(frames[frame_idx, ::5, ::5, [0,1,2]].T)